# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

c:\GoogleDrive\Udacity\Data Engineering\project_2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# Uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# Create a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## QUERY #1

### - Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

The query requires that `artist`, `song` and `length` are selected from a table for specified `session_id` and `item_in_session`. For that we need to create the table `song_id_library` with the following schema:

- `session_id` - INT, PRIMARY KEY
- `item_in_session` - INT, CLUSTERING COLUMN
- `artist` - TEXT
- `song` - TEXT
- `length` - DOUBLE

The SQL query required is as follows:

```SQL
    SELECT artist, song, length, session_id, item_in_session FROM song_id_library WHERE session_id = 338 AND item_in_session = 4
```

In [8]:
query = """CREATE TABLE IF NOT EXISTS song_id_library 
                (session_id int, item_in_session int, artist text, song text, length double,
                 PRIMARY KEY (session_id, item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_id_library (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artist, song, length, session_id, item_in_session FROM song_id_library WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print("Artist:", row.artist, "\t/ Song title:", row.song, "\t/ Length:", row.length, "\t(Session ID:", row.session_id, "\t/ ItemInSession:", row.item_in_session, ")")

Artist: Faithless 	/ Song title: Music Matters (Mark Knight Dub) 	/ Length: 495.3073 	(Session ID: 338 	/ ItemInSession: 4 )


##QUERY #2
### - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The query requires that `artist`, `song`, `first_name` and `last_name` are selected from a table for specified `session_id` and `user_id`. The results also need to be sorted by `item_in_session`. For that we need to create the table song_id_library with the following schema:

- `session_id` - INT, PARTITION KEY
- `user_id` - INT, PARTITION KEY
- `item_in_session` - INT, CLUSTERING COLUMN
- `artist` - TEXT
- `song` - TEXT
- `first_name` - TEXT
- `last_name` - TEXT

The SQL query required is as follows:

```SQL
    SELECT artist, song, first_name, last_name, session_id, user_id, item_in_session FROM song_listened_library WHERE user_id = 10 AND session_id = 182
```

In [11]:
query = """CREATE TABLE IF NOT EXISTS song_listened_library 
                (session_id int, user_id int, item_in_session int, artist text, song text, first_name text, last_name text,
                 PRIMARY KEY ((session_id, user_id), item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listened_library (session_id, user_id, item_in_session, artist, song, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

query = "SELECT artist, song, first_name, last_name, session_id, user_id, item_in_session FROM song_listened_library WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print("Artist:", row.artist, "\t/ Song title:", row.song, "\t/ First Name:", row.first_name, "\t/ Last Name:", row.last_name, "\t(Session ID:", row.session_id, "\t/ User ID:", row.user_id, "\t/ ItemInSession:", row.item_in_session, ")")

Artist: Down To The Bone 	/ Song title: Keep On Keepin' On 	/ First Name: Sylvie 	/ Last Name: Cruz 	(Session ID: 182 	/ User ID: 10 	/ ItemInSession: 0 )
Artist: Three Drives 	/ Song title: Greece 2000 	/ First Name: Sylvie 	/ Last Name: Cruz 	(Session ID: 182 	/ User ID: 10 	/ ItemInSession: 1 )
Artist: Sebastien Tellier 	/ Song title: Kilometer 	/ First Name: Sylvie 	/ Last Name: Cruz 	(Session ID: 182 	/ User ID: 10 	/ ItemInSession: 2 )
Artist: Lonnie Gordon 	/ Song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 	/ First Name: Sylvie 	/ Last Name: Cruz 	(Session ID: 182 	/ User ID: 10 	/ ItemInSession: 3 )


##QUERY #3
### - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The query requires that `first_name` and `last_name` are selected from a table for specified `song`. For that we need to create the table song_id_library with the following schema:

- `song` - TEXT, PARTITION KEY
- `user_id` - INT, PARTITION KEY
- `first_name` - TEXT
- `last_name` - TEXT

The SQL query required is as follows:

```SQL
    SELECT song, user_id, first_name, last_name FROM user_listened_library WHERE song = 'All Hands Against His Own'
```

In [12]:
query = """CREATE TABLE IF NOT EXISTS user_listened_library 
                (song text, user_id int, first_name text, last_name text,
                 PRIMARY KEY ((song, user_id)) )"""
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listened_library (song, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

query = "SELECT song, user_id, first_name, last_name FROM user_listened_library WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print("Song title:\t", row.song, "\t/ First Name:\t", row.first_name, "\t/ Last Name:\t", row.last_name, "\t(User ID:\t", row.user_id, ")")

Song title:	 All Hands Against His Own 	/ First Name:	 Jacqueline 	/ Last Name:	 Lynch 	(User ID:	 29 )
Song title:	 All Hands Against His Own 	/ First Name:	 Tegan 	/ Last Name:	 Levine 	(User ID:	 80 )
Song title:	 All Hands Against His Own 	/ First Name:	 Sara 	/ Last Name:	 Johnson 	(User ID:	 95 )


### Drop the tables before closing out the sessions

In [13]:
query = "DROP TABLE song_id_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [14]:
query = "DROP TABLE song_listened_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [15]:
query = "DROP TABLE user_listened_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()